In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import optimizers
from keras.utils import np_utils
import keras.backend as K
from keras.utils import to_categorical
import keras
from tensorflow.keras.layers import BatchNormalization
import tensorflow.keras.metrics as metrics
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.datasets import mnist
import pandas as pd
(x_train, y_train) , (x_test, y_test) = mnist.load_data()
train_morpho = pd.read_csv("train-morpho.csv")
test_morpho = pd.read_csv("t10k-morpho.csv")



In [3]:
def print_slant_histograms():
    total_df = pd.concat([train_morpho, pd.DataFrame({'label': y_train})], axis=1)
    
    # Assuming you have a pandas DataFrame called 'df' with columns 'slant' and 'y_train'

    # Group the DataFrame by the 'y_train' column
    grouped = total_df.groupby('label')

    # Create a figure and axes for the subplots
    fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(15, 6))

    # Iterate over the groups and create histograms for each category
    for i, (category, group) in enumerate(grouped):
        # Determine the position of the histogram subplot
        row = i // 5
        col = i % 5
        
        # Plot the histogram for the current category
        ax = axes[row, col]
        ax.hist(group['slant'], bins=50)
        ax.set_title(f'{category}')
        ax.set_xlabel('Slant')
        ax.set_ylabel('Frequency')

    # Adjust the layout of subplots
    plt.tight_layout()

    # Show the plot
    plt.show()


In [ ]:
def create_base_MNIST_model():
    K.clear_session()

    earlyStopping = keras.callbacks.EarlyStopping(
        monitor="mae",
        min_delta=0,
        patience=5,
        verbose=0,
        mode="auto",
        baseline=None,
        restore_best_weights=True,
    )


    # Define the input shape
    input_shape = (28, 28, 1)  # Assuming grayscale images, use (28, 28, 3) for RGB images

    # Create the input layer
    inputs = keras.Input(shape=input_shape)

    # Flatten the input
    flatten = layers.Flatten()(inputs)

    # Add one or more hidden layers
    hidden1 = layers.Dense(500, activation='relu')(flatten)
    bn_h1 = layers.BatchNormalization()(hidden1)
    hidden2 = layers.Dense(300, activation='relu')(bn_h1)

    # Add the output layer
    outputs = layers.Dense(1, activation='linear')(hidden2)  # Assuming 10 classes for a multi-class classification task

    # Create the model
    model = keras.Model(inputs=inputs, outputs=outputs)

    # Print the model summary
    model.summary()


    model.compile(loss = 'mean_squared_error', optimizer=optimizers.Adam(learning_rate=0.001, decay=1E-4), metrics=['mae'])


    model.fit(x_train, 
            np.array(train_morpho['slant']),
            epochs=200, batch_size=120, 
            verbose=0, 
            validation_data=(x_test, np.array(test_morpho['slant'])), 
            callbacks=[earlyStopping],
            )

    return model

In [ ]:
def create_embedding_MNIST_model():    
    K.clear_session()

    earlyStopping = keras.callbacks.EarlyStopping(
        monitor="mae",
        min_delta=0,
        patience=5,
        verbose=0,
        mode="auto",
        baseline=None,
        restore_best_weights=True,
    )


    # Define the input shape
    image_input_shape = (28, 28, 1)  # Assuming grayscale images, use (28, 28, 3) for RGB images
    embedding_input_shape = (1,)
    # Create the input layer
    image_input = keras.Input(shape=input_shape)
    # Flatten the image input
    image_flatten = layers.Flatten()(image_input)

    embedding_input = keras.Input(shape=embedding_input_shape)
    embedding_layer = layers.Embedding(10,2)(embedding_input)
    embedding_flatten = layers.Flatten()(embedding_layer)

    # Flatten the input
    concatenated = layers.Concatenate()([image_flatten, embedding_flatten])

    # Add one or more hidden layers
    hidden1 = layers.Dense(500, activation='relu')(concatenated)
    bn_h1 = layers.BatchNormalization()(hidden1)
    hidden2 = layers.Dense(300, activation='relu')(bn_h1)

    # Add the output layer
    outputs = layers.Dense(1, activation='linear')(hidden2)  # Assuming 10 classes for a multi-class classification task

    # Create the model
    model = keras.Model(inputs=[image_input, embedding_input], outputs=outputs)

    # Print the model summary
    model.summary()

    model.compile(loss = 'mean_squared_error', optimizer=optimizers.Adam(learning_rate=0.001, decay=1E-4), metrics=['mae'])

    model.fit([x_train, y_train] , 
          np.array(train_morpho['slant']),
          epochs=200, batch_size=120, 
          verbose=0, 
          validation_data=([x_test, y_test], np.array(test_morpho['slant'])), 
          callbacks=[earlyStopping],
         )

    return model


In [ ]:
def show_embedding_vectors(model):
    embedding_weights = model.layers[2].get_weights()[0]
    indices = np.arange(len(embedding_weights))


    x_coordinates = [vector[0] for vector in embedding_weights]
    y_coordinates = [vector[1] for vector in embedding_weights]
    plt.scatter(x_coordinates, y_coordinates)

    for i, index in enumerate(indices):
        plt.annotate(index, (x_coordinates[i], y_coordinates[i]))

    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title('Embedding Vectors')
    plt.show()

In [ ]:
def create_param_MNIST_model(param):
    K.clear_session()

    earlyStopping = keras.callbacks.EarlyStopping(
    monitor="mae",
    min_delta=0,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
    )

    # Define the input shape
    image_input_shape = (28, 28, 1)  # Assuming grayscale images, use (28, 28, 3) for RGB images
    width_input_shape = (1,)
    # Create the input layer
    image_input = keras.Input(shape=input_shape)
    # Flatten the image input
    image_flatten = layers.Flatten()(image_input)

    width_input = keras.Input(shape=width_input_shape)
    width_flatten = layers.Flatten()(width_input)

    # Flatten the input
    concatenated = layers.Concatenate()([image_flatten, width_flatten])

    # Add one or more hidden layers
    hidden1 = layers.Dense(500, activation='relu')(concatenated)
    bn_h1 = layers.BatchNormalization()(hidden1)
    hidden2 = layers.Dense(300, activation='relu')(bn_h1)

    # Add the output layer
    outputs = layers.Dense(1, activation='linear')(hidden2)  # Assuming 10 classes for a multi-class classification task

    # Create the model
    model = keras.Model(inputs=[image_input, width_input], outputs=outputs)

    # Print the model summary

    model.compile(loss = 'mean_squared_error', optimizer=optimizers.Adam(learning_rate=0.001, decay=1E-4), metrics=['mae'])


    model.fit([x_train, train_morpho[param]] , 
            np.array(train_morpho['slant']),
            epochs=200, batch_size=120, 
            verbose=1, 
            validation_data=([x_test, test_morpho[param]], np.array(test_morpho['slant'])), 
            callbacks=[earlyStopping],
            )
    
    return model